# 4

Look at the [chapter on interactive graphics](https://smart-stats.github.io/ds4bio_book/book/_build/html/interactive.html) and, specifically, the code to display a subject's MRICloud data as a sunburst plot. Do the following. Display this subject's data as a [Sankey diagram](https://plotly.com/python/sankey-diagram/). Display as many levels as you can (at least 3) for Type = 1, starting from the intracranial volume.

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [2]:
# id = 127
subjectData = pd.read_csv('https://raw.githubusercontent.com/smart-stats/ds4bio_book/main/book/assetts/kirby21AllLevels.csv')
subjectData = subjectData.loc[(subjectData.type == 1) & (subjectData.level == 5) & (subjectData.id == 127)]
subjectData = subjectData[['roi', 'volume']]

# Merge the subject data with the multilevel data
multilevel_lookup = pd.read_csv('https://raw.githubusercontent.com/bcaffo/MRIcloudT1volumetrics/master/inst/extdata/multilevel_lookup_table.txt',
                                sep = '\t').drop(['Level5'], axis = 1)
multilevel_lookup.rename(columns={'modify': "roi",
                                  'modify.1': 'level4',
                                  'modify.2': 'level3',
                                  'modify.3': 'level2',
                                  'modify.4': 'level1'}, inplace=True)
multilevel_lookup = multilevel_lookup[['roi', 'level4', 'level3', 'level2', 'level1']]
subjectData = pd.merge(subjectData, multilevel_lookup, on='roi')
subjectData = subjectData.assign(icv = 'ICV')
subjectData = subjectData.assign(comp = subjectData.volume / np.sum(subjectData.volume))

display(subjectData)

,roi,volume,level4,level3,level2,level1,icv,comp
0,SFG_L,12926,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009350
1,SFG_R,10050,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.007270
2,SFG_PFC_L,12783,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.009247
3,SFG_PFC_R,11507,SFG_R,Frontal_R,CerebralCortex_R,Telencephalon_R,ICV,0.008324
4,SFG_pole_L,3078,SFG_L,Frontal_L,CerebralCortex_L,Telencephalon_L,ICV,0.002227
...,...,...,...,...,...,...,...,...
275,Chroid_LVetc_L,444,AnteriorLateralVentricle_L,LateralVentricle_L,Ventricle,CSF,ICV,0.000321
276,Chroid_LVetc_R,371,AnteriorLateralVentricle_R,LateralVentricle_R,Ventricle,CSF,ICV,0.000268
277,IV_ventricle,2700,IV_ventricle,IV_ventricle,Ventricle,CSF,ICV,0.001953
278,ECCL_L,292,inf_DPWM_L,InferiorWM_L,WhiteMatter_L,Telencephalon_L,ICV,0.000211


In [3]:
# Links
subjectData_1 = subjectData[['icv', 'level1', 'comp']]
subjectData_1 = subjectData_1.groupby(['icv', 'level1']).sum().reset_index()
subjectData_1.columns = ['source', 'target', 'value']

subjectData_2 = subjectData[['level1', 'level2', 'comp']]
subjectData_2 = subjectData_2.groupby(['level1', 'level2']).sum().reset_index()
subjectData_2.columns = ['source', 'target', 'value']
links = pd.concat([subjectData_1, subjectData_2])

subjectData_3 = subjectData[['level2', 'level3', 'comp']]
subjectData_3 = subjectData_3.groupby(['level2', 'level3']).sum().reset_index()
subjectData_3.columns = ['source', 'target', 'value']
links = pd.concat([links, subjectData_3])

display(links)

,source,target,value
0,ICV,CSF,0.079417
1,ICV,Diencephalon_L,0.008548
2,ICV,Diencephalon_R,0.008362
3,ICV,Mesencephalon,0.007430
4,ICV,Metencephalon,0.115313
...,...,...,...
53,WhiteMatter_R,AnteriorWM_R,0.066377
54,WhiteMatter_R,CorpusCallosum_R,0.009525
55,WhiteMatter_R,InferiorWM_R,0.041452
56,WhiteMatter_R,LimbicWM_R,0.009914


In [4]:
# Labels
labels = list(pd.unique(links[['source', 'target']].values.ravel('K')))
labels_map = {i: j for j, i in enumerate(labels)}

links['source'] = links['source'].map(labels_map)
links['target'] = links['target'].map(labels_map)

display(links)

,source,target,value
0,0,1,0.079417
1,0,2,0.008548
2,0,3,0.008362
3,0,4,0.007430
4,0,5,0.115313
...,...,...,...
53,28,74,0.066377
54,28,75,0.009525
55,28,76,0.041452
56,28,77,0.009914


In [5]:
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad = 15,
        thickness = 20,
        line = dict(color = 'black', width = 0.5),
        label = labels,
        color = 'blue'
    ),
    link=dict(
        source = links.to_dict(orient='list')['source'],
        target = links.to_dict(orient='list')['target'],
        value = links.to_dict(orient='list')['value'],
    )
)])

fig.update_layout(title_text='id = 127', font_size=10)
fig.show()